In [26]:
%load_ext Cython

In [61]:
import numpy as np
from scipy.sparse.linalg import cg
from pprint import pprint
from numpy import array, zeros, diag, diagflat, dot
from numpy.linalg import inv
import time
from matplotlib import pyplot as plt
from scipy.sparse import diags

In [68]:
def symmetric_array_n(n):
    main_diag = np.linspace(2, 2, num=n, dtype=np.int8)
    lowUp_diag = np.linspace(-1, -1, num = n-1, dtype=np.int8)
    diagonals = [main_diag, lowUp_diag, lowUp_diag]
    A = diags(diagonals, [0, -1, 1]).toarray()
    return A

In [31]:
%%cython

cimport numpy as np
cimport cython
import numpy as np

@cython.boundscheck(False) # turn of bounds-checking for entire function

def jacobi(np.ndarray[double, ndim=2] A, np.ndarray[double, ndim=1] b, np.ndarray[double, ndim=1]  P, double tol=1.0e-3, int max_iter=1000):

    if P is None:
        P = np.zeros(len(A[0]))
        
    cdef int N = len(b)
    cdef np.ndarray x = np.zeros(len(A[0]))
    cdef unsigned int k, i, j
    cdef double val, err, relerr
    
    for k in range(0, max_iter):
        for i in range(0, N):
            val = 0
            for j in range(0, N):                
                if j <> i:
                    val += - (A[i, j]*P[j])
            x[i] = (val + b[i])/A[i, i]
            
        err = np.abs(np.linalg.norm(x - P))
        relerr = err / np.abs((np.linalg.norm(x) + np.finfo(np.float).eps))
        P = x.copy()
        
        if(err < tol or relerr < tol):
            return x
    return x

In [9]:
def gauss_seidel(A, b, P=None, tol=1.0e-3, max_iter=1000):
    timer = time.time()
    
    if P is None:
        P = np.zeros(len(A[0]))
    
    N = len(b)
    x = np.zeros(len(A[0]))
    
    for k in range(0, max_iter):
        for i in range(0, N):
            val = 0
            for j in range(0, N):                
                if j < i:
                    val += - (A[i, j]*x[j])
                if j > i:
                    val += - (A[i, j]*P[j])
            x[i] = (val + b[i])/A[i, i]
                
        err = np.abs(np.linalg.norm(x - P))
        relerr = err / np.abs((np.linalg.norm(x) + np.finfo(np.float).eps))
        P = x.copy()
        
        if(err < tol or relerr < tol):
            timer = time.time() - timer
            return x, timer
    timer = time.time() - timer
    return x, timer

In [10]:
def SOR(A, b, P=None, w=None, tol=1.0e-3, max_iter=1000):
    timer = time.time()
    
    if P is None:
        P = np.zeros(len(A[0]))
        
    if w is None:
        w = 1.0
    
    N = len(b)
    x = np.zeros(len(A[0]))
    
    for k in range(0, max_iter):
        for i in range(0, N):
            val = 0
            for j in range(0, N):                
                if j < i:
                    val += - (A[i, j]*x[j])
                if j > i:
                    val += - (A[i, j]*P[j])
            x[i] = (1-w)*P[i] + w*(val + b[i])/A[i, i]
                
        err = np.abs(np.linalg.norm(x - P))
        relerr = err / np.abs((np.linalg.norm(x) + np.finfo(np.float).eps))
        P = x.copy()
        
        if(err < tol or relerr < tol):
            timer = time.time() - timer
            return x, timer
    timer = time.time() - timer
    return x, timer

In [11]:
def conjugate_gradient(A, b, x=None, max_iter=1000, tol=1.0e-3):
    timer = time.time()
    
    if x is None:
        x = np.matrix(np.zeros(len(A[0]))).transpose()
        
    r0 = b - np.dot(A, x)
    p = r0

#   Start iterations    
    for i in xrange(max_iter):
        a = float(np.dot(r0.T, r0)/np.dot(np.dot(p.T, A), p))
        x = x + p*a
        ri = r0 - np.dot(A*a, p)

        if np.linalg.norm(ri) < tol:
            timer = time.time() - timer
            return x, timer
        b = float(np.dot(ri.T, ri)/np.dot(r0.T, r0))
        p = ri + b * p
        r0 = ri
    timer = time.time() - timer
    return x, timer

In [15]:
#Two Point Boundary Value Problem
def f(x):
    return -24.0*x**2

In [12]:
A, duration = symmetric_array_n(10)
b = np.matrix(array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0])).transpose()
X = np.linalg.lstsq(A, b)[0]

pprint(X)

matrix([[ 20.],
        [ 39.],
        [ 56.],
        [ 70.],
        [ 80.],
        [ 85.],
        [ 84.],
        [ 76.],
        [ 60.],
        [ 35.]])


In [ ]:
eig = np.linalg.eig(A)[0]
print "Eigenvalues: "
pprint(eig)

In [ ]:
n = 999
A, duration = symmetric_array_n(n)
x = np.arange(1.0/(n+1), 1, 1.0/(n+1))
b = np.matrix(np.array(f(x)*(-1.0/n+1)**2))
b[0] = b[0] + 1
b = b.transpose()

ITERATIONS = 500

# Jacobi
sol, duration = jacobi(A,b,max_iter=ITERATIONS)

print "Jacobi:"
print "x:", sol
print ""

print "Duration: "
print duration, "s"
print ""


# Gauss-Seidel
sol, duration = gauss_seidel(A, b, max_iter=ITERATIONS)

print "Gauss-Seidel:"
print "x:", sol
print ""

print "Duration: "
print duration, "s"
print ""

# SOR
sol, duration = SOR(A, b, w = 1.25, max_iter=ITERATIONS)

print "SOR:"
print "x:", sol
print ""

print "Duration: "
print duration, "s"
print ""

# Conjugate Gradient
sol, duration = conjugate_gradient(A, b, max_iter=ITERATIONS)

print "Conjugate-Gradient:"
print "x:", sol
print ""

print "Duration: "
print duration, "s"
print ""

In [73]:
n = 999
A = symmetric_array_n(n)
x = np.arange(1.0/(n+1), 1, 1.0/(n+1))
b = np.array(f(x)*(-1.0/n+1)**2)
b[0] = b[0] + 1
b = b.transpose()
ITERATIONS = 1000

%timeit sol = jacobi(A,b, None, max_iter=ITERATIONS)

1 loops, best of 3: 2.12 s per loop


In [71]:
b.shape

(999,)